In [1]:
import requests
import pandas as pd
import time

url = ('http://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=apple&begin_date='+'2010-01-01'+'&end_date='+'2010-01-01'+'&api-key=108e912b35fd41d2a363fec157044510')
response = requests.get(url)
data = response.json()
#print(data)# Convert response to JSON format
data = pd.DataFrame(data)   # Convert json data to dataframe
data

,status,copyright,response
docs,OK,Copyright (c) 2018 The New York Times Company....,[{'web_url': 'https://bits.blogs.nytimes.com/2...
meta,OK,Copyright (c) 2018 The New York Times Company....,"{'hits': 5, 'offset': 0, 'time': 13}"


In [2]:
#Convert the response column's dictionary into different columns
data['response'].apply(pd.Series)
responseDF = pd.concat([data.drop(['response'], axis=1), data['response'].apply(pd.Series)], axis=1)
responseDF = responseDF.T

#Split the docs column to get the details of each news
responseDF['docs'].apply(pd.Series)
docsDF = pd.concat([responseDF.drop(['docs'], axis=1), responseDF['docs'].apply(pd.Series)], axis=1)
docsDF

# From the headlines column, get the main headline and delete rest of the details
headline = pd.DataFrame(docsDF['headline'])
headline['headline'].apply(pd.Series)
headlineDF = pd.concat([headline.drop(['headline'], axis=1), headline['headline'].apply(pd.Series)], axis=1)

#Join the main headline to docsDF dataframe
docsDF = docsDF.join(headlineDF['main'])

# Remove all the irrelevant rows from dataframe
docsDF = docsDF.drop('status', axis = 0)
docsDF = docsDF.drop('copyright', axis = 0)
docsDF = docsDF.drop('hits', axis = 0)
docsDF = docsDF.drop('offset', axis = 0)
newsData = docsDF.drop('time', axis = 0)

# Filter columns based on data required
newsData = newsData.filter(items=['snippet', 'abstract','source','pub_date',
                                'document_type','news_desk','type_of_material','word_count','score','main'])
newsData.head()


/Users/surbhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/range.py:465: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return self._int64index.union(other)
/Users/surbhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/Users/surbhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/Users/surbhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


,snippet,abstract,source,pub_date,document_type,news_desk,type_of_material,word_count,score,main
0,Here's a look at what might be big in the tech...,Here's a look at what might be big in the tech...,The New York Times,2010-01-01T08:08:15Z,blogpost,NaN,Blog,527.0,0.020397,Five Tech Themes for 2010
1,"As the year drew to a close, linguistic author...","As the year drew to a close, linguistic author...",The New York Times,2010-01-01T05:18:20Z,blogpost,NaN,Blog,633.0,0.008834,Words of 2009 and the 2000s
2,Cartoon versions of Prime Minister Vladimir V....,NaN,The New York Times,2010-01-01T00:00:00Z,article,Foreign,News,1099.0,0.006758,Making a Little Fun of Russia’s Powerful
3,The final week of the regular season features ...,The final week of the regular season features ...,The New York Times,2010-01-01T06:00:39Z,blogpost,NaN,Blog,2300.0,0.003575,Week 17 N.F.L. Matchups
4,Selective listings from theater critics of The...,NaN,The New York Times,2010-01-01T00:00:00Z,article,Weekend,Schedule,2855.0,0.002928,Theater Listings: Jan. 1 — 7


In [13]:
import datetime

start = datetime.datetime.strptime("2014-01-02", "%Y-%m-%d")
end = datetime.datetime.strptime("2015-01-01", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

#for date in date_generated:
    #print(date.strftime("%Y-%m-%d"))
str(date_generated[0]).split(" ")[0]

'2014-01-02'

In [14]:
#get the data from 2014 to 2015

for i in range(0,len(date_generated)):
    d = str(date_generated[i]).split(" ")[0]
    count = 0
    newsDesk = "Business/Financial Desk"
    url = ('http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Apple&fq=Apple&news_desk={0}&begin_date='+d +'&end_date='+d+'&api-key=108e912b35fd41d2a363fec157044510'.format(newsDesk))
    response = requests.get(url)
    print(response)
    data = response.json()    # Convert response to JSON format
    try:
        data = pd.DataFrame(data)
        if 'errors' not in data.columns and 'response' in data.columns:
            data['response'].apply(pd.Series)
            responseDF = pd.concat([data.drop(['response'], axis=1), data['response'].apply(pd.Series)], axis=1)
            responseDF = responseDF.T

            #Split the docs column to get the details of each news
            responseDF['docs'].apply(pd.Series)
            docsDF = pd.concat([responseDF.drop(['docs'], axis=1), responseDF['docs'].apply(pd.Series)], axis=1)
            docsDF

            # From the headlines column, get the main headline and delete rest of the details
            headline = pd.DataFrame(docsDF['headline'])
            headline['headline'].apply(pd.Series)
            headlineDF = pd.concat([headline.drop(['headline'], axis=1), headline['headline'].apply(pd.Series)], axis=1)

            #Join the main headline to docsDF dataframe
            docsDF = docsDF.join(headlineDF['main'])

            # Remove all the irrelevant rows from dataframe
            docsDF = docsDF.drop('status', axis = 0)
            docsDF = docsDF.drop('copyright', axis = 0)
            docsDF = docsDF.drop('hits', axis = 0)
            docsDF = docsDF.drop('offset', axis = 0)
            newsDataIntermediate = docsDF.drop('time', axis = 0)

            # Filter columns based on data required
            newsDataIntermediate = newsDataIntermediate.filter(items=['snippet', 'abstract','source','pub_date',
                                            'document_type','news_desk','type_of_material','word_count','score','main'])
            newsData = newsData.append(newsDataIntermediate, ignore_index=True)
            print(d)
        
    except Exception as e:
        print(d)
        print(e)
        continue
    
    time.sleep(5) 


<Response [200]>
2014-01-02
<Response [200]>
2014-01-03
<Response [200]>
2014-01-04
<Response [200]>
2014-01-05
<Response [200]>
2014-01-06
<Response [200]>
2014-01-07
<Response [200]>
2014-01-08
<Response [200]>
2014-01-09
<Response [200]>
2014-01-10
<Response [200]>
2014-01-11
'headline'
<Response [200]>
2014-01-12
<Response [200]>
2014-01-13
<Response [200]>
2014-01-14
<Response [200]>
2014-01-15
<Response [200]>
2014-01-16
<Response [200]>
2014-01-17
<Response [200]>
2014-01-18
<Response [200]>
2014-01-19
<Response [200]>
2014-01-20
<Response [200]>
2014-01-21
<Response [200]>
2014-01-22
<Response [200]>
2014-01-23
<Response [200]>
2014-01-24
<Response [200]>
2014-01-25
<Response [200]>
2014-01-26
<Response [200]>
2014-01-27
<Response [200]>
2014-01-28
<Response [200]>
2014-01-29
<Response [200]>
2014-01-30
<Response [200]>
2014-01-31
<Response [200]>
2014-02-01
<Response [200]>
2014-02-02
<Response [200]>
2014-02-03
<Response [200]>
2014-02-04
<Response [200]>
2014-02-05
<Response

<Response [429]>
2014-08-27
If using all scalar values, you must pass an index
<Response [429]>
2014-08-28
If using all scalar values, you must pass an index
<Response [429]>
2014-08-29
If using all scalar values, you must pass an index
<Response [429]>
2014-08-30
If using all scalar values, you must pass an index
<Response [429]>
2014-08-31
If using all scalar values, you must pass an index
<Response [200]>
2014-09-01
<Response [200]>
2014-09-02
<Response [200]>
2014-09-03
<Response [200]>
2014-09-04
<Response [200]>
2014-09-05
<Response [200]>
2014-09-06
<Response [200]>
2014-09-07
<Response [200]>
2014-09-08
<Response [200]>
2014-09-09
<Response [200]>
2014-09-10
<Response [200]>
2014-09-11
<Response [200]>
2014-09-12
<Response [200]>
2014-09-13
<Response [200]>
2014-09-14
<Response [200]>
2014-09-15
<Response [200]>
2014-09-16
<Response [200]>
2014-09-17
<Response [200]>
2014-09-18
<Response [200]>
2014-09-19
<Response [200]>
2014-09-20
<Response [200]>
2014-09-21
<Response [200]>


In [15]:
newsData

,abstract,document_type,main,news_desk,pub_date,score,snippet,source,type_of_material,word_count
0,Here's a look at what might be big in the tech...,blogpost,Five Tech Themes for 2010,NaN,2010-01-01T08:08:15Z,0.020397,Here's a look at what might be big in the tech...,The New York Times,Blog,527.0
1,"As the year drew to a close, linguistic author...",blogpost,Words of 2009 and the 2000s,NaN,2010-01-01T05:18:20Z,0.008834,"As the year drew to a close, linguistic author...",The New York Times,Blog,633.0
2,NaN,article,Making a Little Fun of Russia’s Powerful,Foreign,2010-01-01T00:00:00Z,0.006758,Cartoon versions of Prime Minister Vladimir V....,The New York Times,News,1099.0
3,The final week of the regular season features ...,blogpost,Week 17 N.F.L. Matchups,NaN,2010-01-01T06:00:39Z,0.003575,The final week of the regular season features ...,The New York Times,Blog,2300.0
4,NaN,article,Theater Listings: Jan. 1 — 7,Weekend,2010-01-01T00:00:00Z,0.002928,Selective listings from theater critics of The...,The New York Times,Schedule,2855.0
5,NaN,article,"In the Shadows, Day Laborers Left Homeless as ...",Metro,2010-01-02T00:00:00Z,0.007561,"As construction jobs have dried up, immigrants...",The New York Times,News,932.0
6,"In today's Media Equation column, David Carr l...",blogpost,Media Equation: And a Tablet Will Lead Them ...,NaN,2010-01-03T22:08:49Z,0.016419,"In today's Media Equation column, David Carr l...",The New York Times,Blog,189.0
7,NaN,article,A Taste of Eastern Europe,Metro,2010-01-03T00:00:00Z,0.011351,Delicacy International Market has a decidedly ...,The New York Times,News,303.0
8,NaN,article,"In a Pizza Mecca, a Nontraditional Take",Metro,2010-01-03T00:00:00Z,0.007536,Kitchen Zinc serves what is billed as artisana...,The New York Times,Review,818.0
9,NaN,article,Articles of Faith,Magazine,2010-01-03T00:00:00Z,0.007127,The existential crisis of magazines online.,The New York Times,News,946.0


In [17]:
newsData.to_csv("from2015.csv", encoding='utf-8', index=False)